# Post-Mix Analysis Pipeline - Cleaned Up Version

This notebook provides a streamlined end-to-end audio post-production pipeline:
1. Audio import and analysis
2. Pre-mastering preparation 
3. Mastering with different styles
4. Streaming platform simulation
5. Comparison reporting

**Key improvements in this version:**
- Centralized configuration system
- Proper error handling
- Eliminated code duplication
- Consistent naming conventions
- Modular, reusable components

In [1]:
# Import all required modules with cleaned up dependencies
from __future__ import annotations
import sys
import importlib

# Force reload the mastering orchestrator
if 'mastering_orchestrator' in sys.modules:
  importlib.reload(sys.modules['mastering_orchestrator'])

# Re-import
from mastering_orchestrator import MasteringOrchestrator, LocalMasterProvider
print("✅ Modules reloaded with latest changes!")
# Core modules 
import os
import json
import numpy as np
import soundfile as sf
import pandas as pd
import matplotlib.pyplot as plt
from dataclasses import asdict

# Project modules (now cleaned up)
from config import CONFIG, CFG  # New centralized configuration
from audio_utils import validate_audio  # New utilities
from utils import ensure_audio_valid  # Audio validation from utils
from data_handler import *
from analysis import *
from dsp_premitives import *
from processors import *
from render_engine import *
from pre_master_prep import *
from streaming_normalization_simulator import *
from comparison_reporting import *
from presets_recommendations import *
from logging_versioning import *
from utils import *
from mastering_orchestrator import *
from stem_mastering import *  # New stem mastering capabilities

print("✓ All modules loaded successfully with improved error handling and configuration")
print(f"✓ Using configuration version: {CONFIG.to_dict()['audio']}")
print(f"✓ Workspace configured: {CONFIG.workspace.get_workspace_root()}")
print(f"✓ Processing mode configured: {CONFIG.pipeline.get_processing_mode()}")
if CONFIG.pipeline.is_stem_mode():
    print("✓ Stem mastering enabled - 4-category intelligent processing available")
else:
    print("✓ Single-file processing mode - traditional pipeline")

Post-Mix I/O layer loaded: AudioBuffer, load_wav, save_wav, resample_poly, slice_preview, with_suffix, auto_out_path, sha256_file, print_audio_summary.
Convenience helpers loaded: make_workspace, batch_load_wavs, env_fingerprint, Manifest, write/read_manifest, register_input, register_artifact, import_mix.
✅ Modules reloaded with latest changes!
Analysis layer loaded: analyze_wav/analyze_audio_array, analysis_table, plot_spectrum, plot_short_term_loudness, plot_waveform_excerpt, LUFS approx, true-peak approx, stereo & health metrics.
DSP Primitives Layer loaded:
- Gain/level: apply_gain_db, normalize_peak, normalize_lufs, measure_peak, measure_rms
- Filters: highpass_filter, lowpass_filter, bandpass_filter, shelf_filter, peaking_eq, notch_filter, tilt_eq
- Stereo: mid_side_encode, mid_side_decode, stereo_widener
- Dynamics: compressor (soft‑knee), transient_shaper
- Fades: fade_in, fade_out
- K‑weighting/LUFS approx: k_weight, lufs_integrated_approx
Patched: render_from_cache now unpac

## Dual Processing Modes

This pipeline now supports **two processing modes**:

### 🎛️ **Stem Mastering Mode** (NEW!)
- Process 4 stem categories separately: **drums**, **bass**, **vocals**, **music** 
- Each stem gets optimized processing based on its content type
- Intelligent stem summing with category-specific gain staging
- 5 stem combinations: Punchy Mix, Wide & Open, Tight & Controlled, Aggressive, Balanced
- Set `RUN_STEM_MASTERING = True` and configure `STEM_PATHS`

### 📄 **Single-File Mode** (Traditional)
- Process complete stereo mix with dial-based variants
- 38+ processing variants including presets and recommendations
- Genre-specific and style-specific processing options  
- Set `RUN_SINGLE_FILE = True` (default)

Both modes produce organized output folders and comprehensive reports for A/B comparison.

In [2]:
# ============================================
# CONFIGURATION - Edit these values as needed
# ============================================

# ============================================
# PROCESSING MODE SELECTION
# ============================================
# Choose processing modes to run:
# - Set RUN_SINGLE_FILE = True to run traditional single-file processing
# - Set RUN_STEM_MASTERING = True to run 4-stem intelligent processing
# - Set BOTH to True to compare single-file vs stem mastering results!

RUN_SINGLE_FILE = False      # Run traditional single-file processing
RUN_STEM_MASTERING = True   # Run stem mastering processing

# Single file input (used when RUN_SINGLE_FILE = True)
# BASE_INPUT_PATH = "/Users/itay/Documents/post_mix_data/mixes/crashing_stems/"
# BASE_INPUT_PATH = "/Users/itay/Documents/post_mix_data/mixes/pondomonium/"
BASE_INPUT_PATH = "/Users/itay/Documents/post_mix_data/mixing_sessions/session_20250828_181055/stems/"


MIX_SOURCE_PATH = BASE_INPUT_PATH + "mix.wav"
# Stem file paths (only used if RUN_STEM_MASTERING = True)
# STEM_PATHS = {
#     "hats": BASE_INPUT_PATH + "hats.wav",    # Update these paths
#     "bass": BASE_INPUT_PATH + "bass.wav",     # to your actual
#     "music": BASE_INPUT_PATH + "music_no_guitars.wav",    # Set to None if not available
#     "kick": BASE_INPUT_PATH + "kick.wav",
#     "snare": BASE_INPUT_PATH + "snare.wav",
#     "guitar": BASE_INPUT_PATH + "guitar.wav",
#
# }
STEM_PATHS = {
    "drums": BASE_INPUT_PATH + "drums.wav",    # Update these paths
    "bass": BASE_INPUT_PATH + "bass.wav",     # to your actual
    "vocals": BASE_INPUT_PATH + "vocals.wav",   # "/Users/itay/Documents/post_mix_data/stems/vocals.wav",  # stem files
    "music": BASE_INPUT_PATH + "music.wav"}
# ============================================
# STEM BALANCE CONTROL - ADD THIS HERE!
# ============================================
if RUN_STEM_MASTERING:
  from stem_balance_helper import set_stem_balance
  # set_stem_balance(
  #     hats=0.15,
  #     kick = 0.15,
  #     snare = 0.15,
  #     guitar = 0.15,
  #     bass=0.2,
  #     music =.35    # Make music louder
  # )

  set_stem_balance(
      drums=0.9,
      bass=0.8,
      vocals=0.3,
      music=.55    # Make music louder
  )

# Project settings
PROJECT_NAME = "postmix_dual_mode_v1"

# Processing options
ENABLE_STREAMING_SIMULATION = True
MAX_RECOMMENDATIONS = 3
LEVEL_MATCH_LUFS = -14.0  # For A/B comparison previews
REFERENCE_NAME = "Original"

# Validate configuration
print(f"Project: {PROJECT_NAME}")
print(f"🎯 Processing Modes Selected:")
if RUN_SINGLE_FILE:
    print(f"  ✅ Single-file processing: {MIX_SOURCE_PATH}")
if RUN_STEM_MASTERING:
    valid_stems = [k for k, v in STEM_PATHS.items() if v is not None]
    print(f"  ✅ Stem mastering: {len(valid_stems)} stems ({', '.join(valid_stems)})")
if not RUN_SINGLE_FILE and not RUN_STEM_MASTERING:
    print("  ⚠️  No processing modes selected!")

print(f"📁 Workspace root: {CONFIG.workspace.get_workspace_root()}")
print(f"Audio config: {CONFIG.audio.default_bit_depth} @ {CONFIG.audio.render_peak_target_dbfs} dBFS peak target")
print(f"Processing config: Limiter attack={CONFIG.processing.limiter_attack_ms}ms, release={CONFIG.processing.limiter_release_ms}ms")
print(f"\n📍 IMPORTANT: All outputs will be saved to {CONFIG.workspace.get_workspace_root()}")
print("   This keeps your Git repo clean and prevents accidentally committing large audio files!")

if RUN_SINGLE_FILE and RUN_STEM_MASTERING:
    print("\n🆚 DUAL MODE: Both processing types will run for direct comparison!")
    print("   Output folders will be: single_file/ and stem_mastering/")

✅ Updated stem balance:
🎚️  Current Stem Balance Settings:
drums   : 0.90 (+19.0dB) █████████
bass    : 0.80 (+17.9dB) ████████
vocals  : 0.30 (+11.0dB) ███
music   : 0.55 (+14.8dB) █████
kick    : 0.80 (+17.9dB) ████████
snare   : 0.75 (+17.3dB) ███████
hats    : 0.65 (+16.1dB) ██████
backvocals: 0.70 (+16.7dB) ███████
leadvocals: 0.90 (+19.0dB) █████████
guitar  : 0.75 (+17.3dB) ███████
keys    : 0.80 (+17.9dB) ████████
strings : 0.85 (+18.4dB) ████████
Auto-gain compensation: True
Target peak: 0.8 (-1.9 dBFS)
Project: postmix_dual_mode_v1
🎯 Processing Modes Selected:
  ✅ Stem mastering: 4 stems (drums, bass, vocals, music)
📁 Workspace root: /Users/itay/Documents/post_mix_data/PostMixRuns
Audio config: PCM_24 @ -1.2 dBFS peak target
Processing config: Limiter attack=1.0ms, release=50.0ms

📍 IMPORTANT: All outputs will be saved to /Users/itay/Documents/post_mix_data/PostMixRuns
   This keeps your Git repo clean and prevents accidentally committing large audio files!


In [3]:
# ============================================
# MAIN PROCESSING PIPELINE - DUAL MODE
# ============================================

try:
    # 1) Setup workspace and logging
    print("📁 Setting up workspace...")
    workspace_paths = make_workspace(project=PROJECT_NAME)
    manifest = Manifest(project=PROJECT_NAME, workspace=workspace_paths)
    
    logger = RunLogger.start(workspace_paths.root, tag=PROJECT_NAME)
    environment_info = capture_environment()
    logger.log_event("env", {"environment": environment_info})
    
    # Store results from each mode
    single_file_data = None
    stem_set_data = None
    
    # 2) Import and validate audio based on selected modes
    if RUN_SINGLE_FILE:
        print("📄 Importing single mix file...")
        
        if not os.path.exists(MIX_SOURCE_PATH):
            raise FileNotFoundError(f"Audio file not found: {MIX_SOURCE_PATH}")
            
        mix_path = import_mix(workspace_paths, MIX_SOURCE_PATH, alias="mix.wav")
        register_input(manifest, mix_path, alias="mix")
        register_and_log_artifact(manifest, logger, mix_path, kind="input", 
                                 params={"alias": "mix", "mode": "single_file"}, stage="import_mix")
        
        # Load and validate audio
        audio_data, sample_rate = sf.read(mix_path)
        ensure_audio_valid(audio_data, "input mix")
        
        # Print summary
        audio_buffer = load_wav(mix_path)
        print_audio_summary(audio_buffer, "Original Mix (Single-File)")
        
        single_file_data = {
            "audio_data": audio_data,
            "sample_rate": sample_rate,
            "mix_path": mix_path,
            "audio_buffer": audio_buffer
        }
        
        print("✅ Single-file audio imported and validated successfully")
    
    if RUN_STEM_MASTERING:
        print("🎛️ Importing stems for stem mastering...")
        
        # Validate stem paths
        valid_stem_paths = {k: v for k, v in STEM_PATHS.items() if v and os.path.exists(v)}
        if not valid_stem_paths:
            print("❌ No valid stem files found for stem mastering - skipping stem mode")
            RUN_STEM_MASTERING = False
        else:
            print(f"Found {len(valid_stem_paths)} valid stems: {', '.join(valid_stem_paths.keys())}")
            
            # Load stem set
            stem_set = load_stem_set(valid_stem_paths)
            if not validate_stem_set(stem_set):
                print("❌ Stem set validation failed - skipping stem mode")
                RUN_STEM_MASTERING = False
            else:
                # Register stems as inputs
                for stem_type, stem_path in valid_stem_paths.items():
                    imported_stem = import_mix(workspace_paths, stem_path, alias=f"{stem_type}.wav")
                    register_input(manifest, imported_stem, alias=stem_type)
                    register_and_log_artifact(manifest, logger, imported_stem, kind="stem_input", 
                                             params={"alias": stem_type, "stem_type": stem_type, "mode": "stem_mastering"}, stage="import_stems")
                
                stem_set_data = {
                    "stem_set": stem_set,
                    "valid_stem_paths": valid_stem_paths,
                    "sample_rate": getattr(stem_set, list(valid_stem_paths.keys())[0]).sr
                }
                
                print(f"✅ Stems imported successfully: {len(valid_stem_paths)} stems loaded")
    
    if not RUN_SINGLE_FILE and not RUN_STEM_MASTERING:
        raise ValueError("No processing modes selected or available!")
        
except Exception as e:
    print(f"❌ Setup failed: {e}")
    raise

📁 Setting up workspace...
Workspace created at: /Users/itay/Documents/post_mix_data/PostMixRuns/postmix_dual_mode_v1_20250828-224538
🎛️ Importing stems for stem mastering...
Found 4 valid stems: drums, bass, vocals, music
✅ Loaded drums stem: 358.0s
✅ Loaded bass stem: 358.0s
✅ Loaded vocals stem: 358.0s
✅ Loaded music stem: 358.0s
✅ Found 4 active stems: drums, bass, vocals, music
✅ All stems at 44100 Hz
Imported mix → /Users/itay/Documents/post_mix_data/PostMixRuns/postmix_dual_mode_v1_20250828-224538/inputs/drums.wav
Imported mix → /Users/itay/Documents/post_mix_data/PostMixRuns/postmix_dual_mode_v1_20250828-224538/inputs/bass.wav
Imported mix → /Users/itay/Documents/post_mix_data/PostMixRuns/postmix_dual_mode_v1_20250828-224538/inputs/vocals.wav
Imported mix → /Users/itay/Documents/post_mix_data/PostMixRuns/postmix_dual_mode_v1_20250828-224538/inputs/music.wav
✅ Stems imported successfully: 4 stems loaded


In [4]:
# ============================================
# ANALYSIS AND RECOMMENDATIONS - DUAL MODE
# ============================================

try:
    # Store analysis and plans for each mode
    single_file_plan = None
    stem_plan = None
    analysis_results = {}
    
    # 3) Analyze audio for each selected mode
    if RUN_SINGLE_FILE:
        print("📊 Analyzing single-file audio...")
        
        single_analysis = analyze_wav(single_file_data["mix_path"])
        analysis_results["single_file"] = single_analysis
        
        # Log detailed metrics
        logger.log_metrics("analysis_single_file", {
            "sr": single_analysis.sr, 
            "duration_s": single_analysis.duration_s,
            "peak_dbfs": single_analysis.basic["peak_dbfs"],
            "true_peak_dbfs": single_analysis.true_peak_dbfs,
            "rms_dbfs": single_analysis.basic["rms_dbfs"],
            "lufs_integrated": single_analysis.lufs_integrated,
            "crest_db": single_analysis.basic["crest_db"],
            "bass_energy_%": single_analysis.bass_energy_pct,
            "air_energy_%": single_analysis.air_energy_pct,
            "processing_mode": "single_file"
        })
        
        # Generate processing recommendations
        print("🎯 Generating single-file recommendations...")
        single_recommendations = recommend_from_analysis(single_analysis)
        print("SINGLE-FILE " + recommendation_summary(single_recommendations))
        
        # Build comprehensive processing plan
        single_file_plan = build_all_variants_plan(
            single_recommendations, prefix="SF", top_recommendations=MAX_RECOMMENDATIONS
        )
        
        print(f"✅ Generated {len(single_file_plan)} single-file variants")
    
    if RUN_STEM_MASTERING:
        print("📊 Analyzing stem audio...")
        
        # Create temporary mix for analysis by summing stems
        first_stem = getattr(stem_set_data["stem_set"], list(stem_set_data["valid_stem_paths"].keys())[0])
        temp_mix_path = os.path.join(workspace_paths.inputs, "temp_stem_sum.wav")
        stem_audio_sum = np.zeros((int(stem_set_data["stem_set"].get_total_duration() * first_stem.sr), 2), dtype=np.float32)
        
        for stem_type in stem_set_data["stem_set"].get_active_stems():
            stem_audio = getattr(stem_set_data["stem_set"], stem_type)
            if stem_audio:
                # Convert to stereo if mono
                audio_array = stem_audio.samples
                if audio_array.ndim == 1:
                    audio_array = np.column_stack([audio_array, audio_array])
                
                # Add to sum (with length matching)
                audio_len = min(len(audio_array), len(stem_audio_sum))
                stem_audio_sum[:audio_len] += audio_array[:audio_len] * 0.25  # Scale down to avoid clipping
        
        # Save temporary sum for analysis
        sf.write(temp_mix_path, stem_audio_sum, first_stem.sr, subtype="FLOAT")
        stem_analysis = analyze_wav(temp_mix_path)
        analysis_results["stem_mastering"] = stem_analysis
        
        # Log analysis with stem info
        logger.log_metrics("analysis_stem_sum", {
            "sr": stem_analysis.sr, 
            "duration_s": stem_analysis.duration_s,
            "active_stems": stem_set_data["stem_set"].get_active_stems(),
            "total_stems": len(stem_set_data["stem_set"].get_active_stems()),
            "peak_dbfs": stem_analysis.basic["peak_dbfs"],
            "lufs_integrated": stem_analysis.lufs_integrated,
            "processing_mode": "stem_mastering"
        })
        
        # Use configured stem combinations
        stem_plan = CONFIG.pipeline.stem_combinations
        print(f"✅ Using {len(stem_plan)} stem combination variants")
        print(f"🎛️ Stem combinations: {', '.join([combo[0] for combo in stem_plan])}")
    
    # Log processing plans
    logger.log_params("processing_plans", {
        "single_file_plan_size": len(single_file_plan) if single_file_plan else 0,
        "stem_plan_size": len(stem_plan) if stem_plan else 0,
        "modes_active": [mode for mode in ["single_file", "stem_mastering"] if globals()[f"RUN_{mode.upper()}"]],
    }, code_versions={"presets_recs": CODE_VERSIONS["presets_recs"]})
    
    print("✅ Analysis completed for all selected modes")
    
except Exception as e:
    print(f"❌ Analysis failed: {e}")
    raise

📊 Analyzing stem audio...
✅ Using 34 stem combination variants
🎛️ Stem combinations: Stem_PunchyMix, Stem_WideAndOpen, Stem_TightAndControlled, Stem_Aggressive, Stem_Balanced, Stem_RadioReady, Stem_LiveBand, Stem_EDM_Club, Stem_Intimate, Stem_Experimental, Stem_VintageSoul, Stem_ModernPop, Stem_HeavyRock, Stem_3D_Immersive, Stem_Cinematic_AI, Stem_Binaural_Psycho, Stem_VR_Experience, Stem_Quantum_Club, Stem_Neural_Trance, Stem_Breakbeat_Morph, Stem_Subliminal, Stem_Depth_Natural, Stem_Depth_Dramatic, Stem_Depth_Intimate, Stem_Depth_Stadium, Stem_Depth_VocalFocus, Stem_Musical_Balanced, Stem_Musical_VocalForward, Stem_Musical_Warm, Stem_Musical_Clear, Stem_Musical_Polished, Stem_RadioReady_Depth, Stem_Aggressive_Depth, Stem_PunchyMix_Depth
✅ Analysis completed for all selected modes


In [5]:
# ============================================
# PRE-MASTERING PROCESSING - DUAL MODE
# ============================================

try:
    # Store results from each mode
    all_variant_metadata = []
    
    render_options = RenderOptions(
        target_peak_dbfs=CONFIG.audio.render_peak_target_dbfs, 
        bit_depth=CONFIG.audio.default_bit_depth, 
        hpf_hz=None, 
        save_headroom_first=False
    )
    
    # 5a) Single-file processing
    if RUN_SINGLE_FILE and single_file_plan:
        print("🔧 Single-file pre-mastering preparation...")
        
        # Setup single-file render engine
        sf_engine = RenderEngine(single_file_data["audio_data"], single_file_data["sample_rate"], 
                                 preprocess=PreprocessConfig(
                                     low_cutoff=CONFIG.audio.prep_hpf_hz,
                                     kick_lo=CONFIG.audio.kick_freq_low,
                                     kick_hi=CONFIG.audio.kick_freq_high
                                 ))
        sf_preprocess_metadata = sf_engine.preprocess()
        logger.log_params("single_file_preprocess_cache", sf_preprocess_metadata, 
                         code_versions={
                             "processors": CODE_VERSIONS["processors"], 
                             "render_engine": CODE_VERSIONS["render_engine"]
                         })
        
        # Generate single-file variants in dedicated subfolder
        sf_variant_dir = os.path.join(workspace_paths.outputs, "single_file", "premasters")
        sf_variant_metadata = sf_engine.commit_variants(sf_variant_dir, single_file_plan, opts=render_options)
        
        # Tag metadata to identify processing mode
        for meta in sf_variant_metadata:
            meta["processing_mode"] = "single_file"
            register_and_log_artifact(manifest, logger, meta["out_path"], 
                                     kind="sf_premaster", params=meta, 
                                     stage=f"single_file__{os.path.basename(meta['out_path'])}")
        
        all_variant_metadata.extend(sf_variant_metadata)
        print(f"✅ Created {len(sf_variant_metadata)} single-file premaster variants")
    
    # 5b) Stem mastering processing
    if RUN_STEM_MASTERING and stem_plan:
        print("🎛️ Stem mastering preparation...")
        
        # Setup stem render engine
        stem_engine = StemRenderEngine(stem_set_data["stem_set"], 
                                     preprocess=PreprocessConfig(
                                         low_cutoff=CONFIG.audio.prep_hpf_hz,
                                         kick_lo=CONFIG.audio.kick_freq_low,
                                         kick_hi=CONFIG.audio.kick_freq_high
                                     ))
        stem_preprocess_metadata = stem_engine.preprocess_all_stems()
        logger.log_params("stem_preprocess_cache", stem_preprocess_metadata, 
                         code_versions={
                             "stem_mastering": CODE_VERSIONS.get("stem_mastering", "1.0"),
                             "render_engine": CODE_VERSIONS["render_engine"]
                         })
        
        # Process stem combinations in dedicated subfolder
        stem_variant_dir = os.path.join(workspace_paths.outputs, "stem_mastering", "premasters")
        stem_results = stem_engine.commit_stem_variants(
            stem_variant_dir, stem_plan, opts=render_options
        )
        
        # Convert stem results to compatible format
        stem_variant_metadata = []
        for variant_name, variant_data in stem_results.items():
            stem_meta = {
                "out_path": variant_data["final_mix_path"],
                "variant_name": variant_name,
                "stem_results": variant_data["stem_results"],
                "active_stems": variant_data["active_stems"],
                "processing_mode": "stem_mastering"
            }
            stem_variant_metadata.append(stem_meta)
            register_and_log_artifact(manifest, logger, variant_data["final_mix_path"], 
                                     kind="stem_premaster", params=variant_data, 
                                     stage=f"stem_mastering__{variant_name}")
        
        all_variant_metadata.extend(stem_variant_metadata)
        print(f"✅ Created {len(stem_variant_metadata)} stem-processed variants")
        print(f"🎛️ Each variant intelligently combined {len(stem_set_data['stem_set'].get_active_stems())} stems")
    
    if not all_variant_metadata:
        raise ValueError("No variants were generated from any processing mode!")
    
    total_single_file = len([v for v in all_variant_metadata if v.get("processing_mode") == "single_file"])
    total_stem = len([v for v in all_variant_metadata if v.get("processing_mode") == "stem_mastering"])
    
    print(f"\n🎯 PRE-MASTERING SUMMARY:")
    print(f"   📄 Single-file variants: {total_single_file}")
    print(f"   🎛️ Stem mastering variants: {total_stem}")
    print(f"   📊 Total variants for mastering: {len(all_variant_metadata)}")
    
    # Store for next step
    variant_metadata = all_variant_metadata

except Exception as e:
    print(f"❌ Pre-mastering failed: {e}")
    raise

# ============================================
# MASTERING STEP - USING ORCHESTRATOR (CREATES FOLDERS!)
# ============================================

try:
    print("\n🎭 Running mastering orchestration with PROPER FOLDER STRUCTURE...")
    
    # Create mastering orchestrator
    orchestrator = MasteringOrchestrator(workspace_paths, manifest)
    provider = LocalMasterProvider(bit_depth="PCM_24")
    
    # Process all premasters using the orchestrator
    master_results = []
    all_master_paths = []
    
    print(f"📂 Found {len(variant_metadata)} premaster files to master")
    
    for variant_meta in variant_metadata:
        premaster_path = variant_meta["out_path"]
        
        if not os.path.exists(premaster_path):
            print(f"⚠️ Skipping missing file: {premaster_path}")
            continue
            
        print(f"\n🎭 Mastering: {os.path.basename(premaster_path)}")
        
        # Use the orchestrator to create proper folder structure
        results = orchestrator.run(
            premaster_path=premaster_path,
            providers=[provider],
            styles=None,  # Uses default 4 styles
            out_tag="masters",
            level_match_preview_lufs=-14.0
        )
        
        # Collect all result paths
        for result in results:
            master_results.append(result)
            all_master_paths.append(result.out_path)
            
            # Also add the -14 LUFS versions (they're created automatically)
            base_dir = os.path.dirname(result.out_path)
            lufs_file = os.path.join(base_dir, f"{result.style}_-14LUFS.wav")
            if os.path.exists(lufs_file):
                all_master_paths.append(lufs_file)
    
    print(f"\n✅ Mastering completed successfully!")
    print(f"📊 Results:")
    print(f"   🎯 Master folders: {len(variant_metadata)}")
    print(f"   📁 Total audio files: {len(all_master_paths)}")
    
    # Show folder structure
    masters_dir = os.path.join(workspace_paths.outputs, "masters")
    if os.path.exists(masters_dir):
        folders = [f for f in os.listdir(masters_dir) if os.path.isdir(os.path.join(masters_dir, f))]
        print(f"\n📁 Master folders created:")
        for folder in sorted(folders):
            folder_path = os.path.join(masters_dir, folder)
            file_count = len([f for f in os.listdir(folder_path) if f.endswith('.wav')])
            print(f"   • {folder}/ ({file_count} files)")
    
    # Update master_paths for later use
    master_paths = all_master_paths
    
    print(f"\n🎊 MASTERS ARE NOW FOLDERS!")
    print(f"Each master is a folder containing 8 audio files (4 styles × 2 versions)")

except Exception as e:
    print(f"❌ Mastering orchestration failed: {e}")
    raise

🎛️ Stem mastering preparation...
✅ Found 4 active stems: drums, bass, vocals, music
✅ All stems at 44100 Hz


KeyboardInterrupt: 

In [ ]:
# ============================================
# SKIP ALL REPORTING - AUDIO FILES ONLY
# ============================================

print("⚡ Skipping all reporting steps - focusing on audio file creation only")
print("✅ Audio processing complete - ready for listening tests")
print("📁 All files available in workspace for immediate use")

# No streaming simulation
# No comparison reports  
# No heavy analysis
# Just the audio files you need

In [ ]:
# ============================================
# MINIMAL SUMMARY - FILES ONLY
# ============================================

print("📁 FINAL SUMMARY - AUDIO FILES READY")
print("="*50)
print(f"Workspace: {workspace_paths.root}")

total_single_file = len([v for v in variant_metadata if v.get("processing_mode") == "single_file"])
total_stem = len([v for v in variant_metadata if v.get("processing_mode") == "stem_mastering"])

if RUN_SINGLE_FILE:
    print(f"\n📄 Single-File: {total_single_file} variants")
    print(f"   📁 {workspace_paths.root}/outputs/single_file/")

if RUN_STEM_MASTERING:
    print(f"\n🎛️ Stem Mastering: {total_stem} variants") 
    print(f"   📁 {workspace_paths.root}/outputs/stem_mastering/")

print(f"\n🎧 READY FOR A/B TESTING!")
print("⚡ No reports = faster workflow!")

In [ ]:
# ============================================
# FINALIZATION AND SUMMARY - DUAL MODE
# ============================================

try:
    # 7) Create reproducibility bundle
    print("📦 Creating reproducibility bundle...")
    
    bundle_path = os.path.join(workspace_paths.reports, "bundles", f"{logger.run_id}.zip")
    
    # Include all outputs from both processing modes
    all_outputs = list(set(master_paths))
    
    reproducibility_zip = make_repro_zip(
        bundle_path,
        workspace_root=workspace_paths.root,
        run_logger=logger,
        env_info=environment_info,
        inputs=([single_file_data["mix_path"]] if single_file_data else []) + 
               (list(stem_set_data["valid_stem_paths"].values()) if stem_set_data else []),
        outputs=all_outputs,
        extra_jsons={
            "code_versions": CODE_VERSIONS, 
            "processing_modes": {
                "single_file_enabled": RUN_SINGLE_FILE,
                "stem_mastering_enabled": RUN_STEM_MASTERING,
                "single_file_variants": len([v for v in variant_metadata if v.get("processing_mode") == "single_file"]),
                "stem_variants": len([v for v in variant_metadata if v.get("processing_mode") == "stem_mastering"])
            },
            "configuration": CONFIG.to_dict(),
            "pipeline_version": "3.0_dual_mode",
            "folder_structure": {
                "single_file": "single_file/premasters/ → single_file/masters/",
                "stem_mastering": "stem_mastering/premasters/ → stem_mastering/masters/",
                "note": "Organized by processing mode for easy comparison"
            }
        },
        readme_text="Dual-mode post-mix pipeline with both single-file and stem mastering capabilities. Results organized by processing mode for direct comparison."
    )
    
    register_and_log_artifact(manifest, logger, reproducibility_zip, 
                             kind="bundle", params={"run_id": logger.run_id, "version": "3.0"}, 
                             stage="repro_zip")
    
    # 8) Write final summary
    logger.write_summary({
        "pipeline_version": "3.0_dual_mode",
        "project": PROJECT_NAME,
        "run_id": logger.run_id,
        "processing_modes": {
            "single_file": RUN_SINGLE_FILE,
            "stem_mastering": RUN_STEM_MASTERING
        },
        "inputs": {
            "single_file": single_file_data["mix_path"] if single_file_data else None,
            "stems": list(stem_set_data["valid_stem_paths"].values()) if stem_set_data else []
        },
        "outputs": {
            "total_variants": len(variant_metadata),
            "total_masters": len(master_paths),
            "single_file_variants": len([v for v in variant_metadata if v.get("processing_mode") == "single_file"]),
            "stem_variants": len([v for v in variant_metadata if v.get("processing_mode") == "stem_mastering"])
        },
        "configuration_snapshot": CONFIG.to_dict(),
        "environment_hash": json_sha256(environment_info)
    })
    
    write_manifest(manifest)
    
    print("\\n" + "="*60)
    print("🎉 DUAL-MODE PROCESSING COMPLETED SUCCESSFULLY!")
    print("="*60)
    print(f"📁 Workspace: {workspace_paths.root}")
    
    total_single_file = len([v for v in variant_metadata if v.get("processing_mode") == "single_file"])
    total_stem = len([v for v in variant_metadata if v.get("processing_mode") == "stem_mastering"])
    
    if RUN_SINGLE_FILE:
        print(f"\\n📄 Single-File Processing:")
        print(f"   Input: {single_file_data['mix_path'] if single_file_data else 'None'}")
        print(f"   Variants: {total_single_file}")
        print(f"   Location: single_file/")
    
    if RUN_STEM_MASTERING:
        print(f"\\n🎛️ Stem Mastering:")
        if stem_set_data:
            print(f"   Stems: {', '.join(stem_set_data['valid_stem_paths'].keys())}")
        print(f"   Variants: {total_stem}")
        print(f"   Location: stem_mastering/")
    
    print(f"\\n📊 Total Results:")
    print(f"   Variants: {len(variant_metadata)}")
    print(f"   Masters: {len(master_paths)}")
    print(f"   Organized in: {len(variant_metadata)} variant subfolders")
    
    print(f"\\n📦 Bundle: {reproducibility_zip}")
    print("\\n✨ Ready for A/B comparison between processing modes!")
    
except Exception as e:
    print(f"❌ Finalization failed: {e}")
    raise

In [ ]:
# This cell has been removed - mastering is now integrated into cell-6

In [ ]:
## Summary of Improvements

This cleaned up version includes:

### ✅ **Code Quality Improvements**
- Removed duplicate mastering provider classes
- Consolidated audio conversion logic into `audio_utils.py`
- Fixed naming inconsistencies throughout codebase
- Eliminated redundant functions and utilities

### ✅ **Configuration Management**
- Centralized configuration system in `config.py`
- Environment variable support for key settings
- Consistent parameter usage across all modules
- Configuration versioning and snapshots

### ✅ **Error Handling**
- Proper exception handling with specific error types
- Input validation for all audio processing functions
- Graceful failure recovery where possible
- Detailed error messages for debugging

### ✅ **Architecture Improvements**
- Simplified over-engineered abstractions
- Cleaner separation of concerns
- More maintainable and readable code structure
- Better documentation and type hints

### 🎛️ **NEW: Stem Mastering Integration**
- **Dual-mode processing**: Choose between single-file or 4-stem processing
- **Intelligent stem processing**: Category-specific dial optimization (drums, bass, vocals, music)
- **Smart stem summing**: Automatic gain staging and bus compression
- **5 stem combinations**: Punchy Mix, Wide & Open, Tight & Controlled, Aggressive, Balanced
- **Backward compatibility**: Existing single-file pipeline unchanged

### 🔧 **Next Steps for Further Improvement**
- Add unit tests for all modules
- Implement async processing for better performance  
- Add more streaming platform profiles
- Create CLI interface for batch processing
- Add real-time monitoring and progress bars
- Expand stem categories and processing variants